In [1]:
import pandas as pd
import data_work 

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick


In [2]:
df = data_work.load_data()
df.head()

,contract_id,contract_start,contract_term,deposit,monthly_amt,company,sector,industry,email,address,city,state,zip
customer_id,,,,,,,,,,,,,
1,41,5/31/19,24,"$86,000.00","$11,200.00",Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,358,6/18/19,13,"$74,000.00","$19,000.00",Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,400,8/11/19,19,"$38,000.00","$8,800.00",Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,424,3/13/19,15,"$77,000.00","$14,300.00",Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,438,10/16/19,22,"$39,000.00","$20,000.00",Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170


In [3]:
df = data_work.clean_values(df)
df.head()

,contract_id,contract_start,contract_term,deposit,monthly_amt,company,sector,industry,email,address,city,state,zip
customer_id,,,,,,,,,,,,,
1,41,5/31/19,24,86000.0,11200.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,358,6/18/19,13,74000.0,19000.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,400,8/11/19,19,38000.0,8800.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,424,3/13/19,15,77000.0,14300.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170
1,438,10/16/19,22,39000.0,20000.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170


In [4]:
df = data_work.summary_calculations(df)
df.head()

,contract_id,contract_start,contract_term,deposit,monthly_amt,company,sector,industry,email,address,city,state,zip,monthly_total_revenue,total_revenue
customer_id,,,,,,,,,,,,,,,
1,41,5/31/19,24,86000.0,11200.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170,268800.0,354800.0
1,358,6/18/19,13,74000.0,19000.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170,247000.0,321000.0
1,400,8/11/19,19,38000.0,8800.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170,167200.0,205200.0
1,424,3/13/19,15,77000.0,14300.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170,214500.0,291500.0
1,438,10/16/19,22,39000.0,20000.0,Bubblebox,Capital Goods,Military/Government/Technical,kyerson0@bizjournals.com,76 Randy Circle,San Diego,CA,92170,440000.0,479000.0


In [5]:
df_detail = data_work.detail_calculations(df)
df_detail.head()

ValueError: time data '5/31/19' does not match format '%m/%d/%Y' (match)

In [6]:
df_detail.tail()

NameError: name 'df_detail' is not defined

In [7]:
df_detail['amt_due'].plot()

NameError: name 'df_detail' is not defined

In [ ]:
df_detail.loc[df_detail['deposit'], 'amt_due'].plot()

In [ ]:
df_detail.loc[(
    (df_detail['deposit'] == False) &
    (df_detail['contract_id'] == 974)
), 'amt_due'].plot()

In [ ]:
df_detail.loc[(
    (df_detail['deposit'] == False) &
    (df_detail['industry'] == 'Television Services')
), 'amt_due'].plot()

In [ ]:
df_detail.groupby('state').agg({'amt_due':'sum'}).plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_detail.groupby('state', as_index=False).agg({'amt_due':'sum'}).plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_detail.groupby('state', as_index=False).agg({'amt_due':'sum'}).sort_values('amt_due').plot.bar(y = 'amt_due', x= 'state')

In [ ]:
df_v = df_detail.reset_index()
df_v['year'] = df_v['due_date'].dt.year
df_v['month'] = df_v['due_date'].dt.month
df_v.head()

In [ ]:
df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot()

In [ ]:
df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot(figsize=(20, 10))

In [ ]:
fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

df_v.groupby(['year', 'month']).agg({'amt_due':sum}).plot(figsize=(20, 10)).yaxis.set_major_formatter(tick)
plt.xticks(rotation=45)